In [ ]:
!pip install mlflow pyngrok
!pip install vaderSentiment
!pip install fastapi uvicorn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.7/629.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/7

In [ ]:
'''Adding a Model to MLflow'''
import mlflow
import mlflow.keras
from datetime import datetime

# Mount Google Drive
#drive.mount('/content/drive')
def log_model(model):
      # Set MLflow tracking URI to Google Drive or a persistent directory
      mlflow.set_tracking_uri("/content/mlruns")

      # Evaluate the model and capture metrics
      #loss= model.evaluate(X_test, y_test, verbose=0)  # Add other metrics if available
      #matrics are added in the compiler

      # Start a new MLflow run
      with mlflow.start_run() as run:
          # Add metadata (timestamp)
          timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
          mlflow.log_param("run_timestamp", timestamp)

          # Log metrics
          #mlflow.log_metric("loss", loss)
          #mlflow.log_metric("accuracy", accuracy)

          # Log the model with a unique name
          model_name = f"model"
          mlflow.keras.log_model(model, artifact_path=model_name)

          # Print the run ID and model name
          print(f"Model logged with name: {model_name}")
          print(f"Run ID: {run.info.run_id}")
          #print(f"Loss: {loss}")#, Accuracy: {accuracy}


In [ ]:
'''Utilisation de Vader pour le sentiment score'''
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def sentiment_scores(tweet):
      # Initialize the SentimentIntensityAnalyzer
      sia = SentimentIntensityAnalyzer()
      #print(sia.polarity_scores(tweet))
      return sia.polarity_scores(tweet)['compound']

print(sentiment_scores("I don't know what to say, it just bad"))


-0.5423


In [ ]:
'''Serveur d'Entrainement et de Prédiction'''
from fastapi import FastAPI
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Layer, Input, LSTM, Dense, Dropout
from tensorflow.keras.models import Model
from keras.utils import get_custom_objects
from fastapi import FastAPI, HTTPException
import tensorflow as tf
from io import StringIO
import os
import mlflow
import mlflow.keras
from mlflow.tracking import MlflowClient
from datetime import timedelta


from pydantic import BaseModel
from typing import List, Dict, Any


app = FastAPI()
# Initialize the model and scaler to avoid reloading it for each prediction
from tensorflow.keras.saving import register_keras_serializable

from keras.layers import Layer, Dense
import tensorflow as tf
from keras.saving import register_keras_serializable

@register_keras_serializable(package='Custom')
class Attention(Layer):
    def __init__(self, **kwargs):
        # Ensure the parent class handles all standard arguments like `trainable`
        super(Attention, self).__init__(**kwargs)
        self.W = Dense(50, use_bias=False)
        self.V = Dense(1, use_bias=False)

    def call(self, hidden_states):
        # Compute attention scores
        score = self.V(tf.nn.tanh(self.W(hidden_states)))
        attention_weights = tf.nn.softmax(score, axis=1)

        # Compute the context vector
        context_vector = tf.reduce_sum(attention_weights * hidden_states, axis=1)
        return context_vector, attention_weights

    def get_config(self):
        # Serialize the layer properly
        config = super(Attention, self).get_config()
        return config

    @classmethod
    def from_config(cls, config):
        # Ensure the `trainable` argument is handled correctly
        return cls(**config)


def sentiment_scores(tweet):
    sid = SentimentIntensityAnalyzer()
    return sid.polarity_scores(tweet)['compound']
    # Apply the sentiment_scores function to each tweet


    return df

def map_sentiment(score):
    if score > 0.05:  # Slightly positive or higher
        return "positive"
    elif score < -0.05:  # Slightly negative or lower
        return "negative"
    else:
        return "neutral"  # Neutral sentiment

def preprocess_data(social_data, stock_data):
    # Preprocess the social data and stock data
    try:
        # Compute sentiment scores
        social_data['sentiment_scores'] = social_data['tweet'].apply(sentiment_scores)
        social_data['sentiment'] = social_data['sentiment_scores'].apply(map_sentiment)


        social_data['time'] = pd.to_datetime(social_data['time'], unit='ms')
        stock_data['time'] = pd.to_datetime(stock_data['time'], unit='ms')


        # Sort both datasets by time
        social_data = social_data.sort_values(by='time')
        stock_data = stock_data.sort_values(by='time')


        # Merge using `merge_asof` with a time tolerance (e.g., 10 day)
        data = pd.merge_asof(stock_data, social_data, on='time', tolerance=pd.Timedelta('60D'), direction='nearest')

        data['followers'] = pd.to_numeric(data['followers'], errors='coerce')  # Converts non-numeric to NaN
        data['sentiment_scores'] = pd.to_numeric(data['sentiment_scores'], errors='coerce')
        data['stock_price'] = data['stock_price'].apply(lambda x: round(x, 3) if np.isfinite(x) else x)
        data['followers'] = data['followers'].apply(lambda x: round(x, 3) if np.isfinite(x) else x)
        data = data.dropna()
        print("The data")
        print(data)

        if data.empty:
            raise ValueError("Merged data is empty. Ensure 'time' columns have matching values in both datasets.")

        # Add weighted sentiment feature
        data['weighted_sentiment'] = data['followers'] * data['sentiment_scores']

        # Select features and target
        features = ['weighted_sentiment']
        target = 'stock_price'

        # Check if required columns exist
        missing_columns = [col for col in features + [target] if col not in data.columns]
        if missing_columns:
            raise ValueError(f"Missing columns in data: {missing_columns}")

        # Check if data is non-empty for scaling
        if data[features + [target]].shape[0] == 0:
            raise ValueError("No samples available for scaling.")

        # Normalize features and target
        data_complete = data.copy()
        scaler = MinMaxScaler()
        data[features + [target]] = scaler.fit_transform(data[features + [target]])
        data_complete = data_complete.to_dict(orient='records')
        return data, features, target, data_complete

    except Exception as e:
        print(f"Error during preprocessing: {e}")
        raise

def create_sequences(data, seq_length):
    X, y = [], []
    print("create_sequence")
    print(data)
    print(seq_length)
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :-1])  # Features (weighted_sentiment)
        y.append(data[i+seq_length, -1])    # Target (stock_price)
    return np.array(X), np.array(y)

def build_and_train_model(X_train, y_train):
    print(X_train)
    print(y_train)
    # Build the LSTM model with attention mechanism
    #X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))  # Reshape for LSTM input
    inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))  # (seq_length, 1)
    lstm_out = LSTM(50, activation='relu', return_sequences=True)(inputs)
    lstm_out = Dropout(0.2)(lstm_out)

    # Apply the custom attention layer
    context_vector, attention_weights = Attention()(lstm_out)

    # Dense layers
    dense_out = Dense(64, activation='relu')(context_vector)
    dense_out = Dropout(0.2)(dense_out)
    output = Dense(1)(dense_out)

    # Define the model
    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss='mse')

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32)#, validation_data=(X_test, y_test)
    return model

@app.post("/train_model")
async def train_model(request: dict):
    # Deserialize the JSON back to DataFrame
    print(request)
    # Extract stock_data and social_data directly from the request dictionary
    stock_data = pd.DataFrame(request['stock_data']['data'], columns=request['stock_data']['columns'])
    social_data = pd.DataFrame(request['social_data']['data'], columns=request['social_data']['columns'])

    # Preprocess data
    data, features, target, data_complete = preprocess_data(social_data, stock_data)

    # Create sequences for LSTM
    seq_length = 5  # Number of timesteps
    data_array = data[features + [target]].values
    X, y = create_sequences(data_array, seq_length)

    # Train-test split
    #split = int(0.8 * len(X))
    #X_train, X_test = X[:split], X[split:]
    #y_train, y_test = y[:split], y[split:]
    print("X")
    print(X)
    print(y)
    X_train = X
    y_train = y
    get_custom_objects().update({"Attention": Attention})
    model = build_and_train_model(X_train, y_train)#, X_test, y_test)
    log_model(model)
    print(data_complete)
    return {"message": "Model trained successfully!", "complete_data" : data_complete}

@app.post("/predict")
async def predict(request: dict):
    #try:
        # Process input data
        social_data = pd.DataFrame(request['social_data']['data'], columns=request['social_data']['columns'])
        print('from source')
        print(social_data['time'])
        social_data['time'] = pd.to_datetime(social_data['time'], unit='ms')

        social_data = social_data.sort_values(by='time')

        social_data['sentiment_scores'] = social_data['tweet'].apply(sentiment_scores)
        social_data['sentiment'] = social_data['sentiment_scores'].apply(map_sentiment)
        social_data['followers'] = pd.to_numeric(social_data['followers'], errors='coerce')  # Converts non-numeric to NaN
        social_data['sentiment_scores'] = pd.to_numeric(social_data['sentiment_scores'], errors='coerce')
        social_data['followers'] = social_data['followers'].apply(lambda x: round(x, 3) if np.isfinite(x) else x)
        social_data = social_data.dropna()
        social_data['weighted_sentiment'] = social_data['followers'] * social_data['sentiment_scores']

        # Normalize the data
        scaler = MinMaxScaler()
        social_data[['weighted_sentiment']] = scaler.fit_transform(social_data[['weighted_sentiment']])

        # Prepare input for prediction
        last_30_days = social_data[-5:]

        # Ensure you are selecting the last 5 values from the 'weighted_sentiment' column
        X_pred = last_30_days[['weighted_sentiment']].values[-5:].reshape(1, 5, 1)

        # If the sequence length exceeds 5, retain only the last 5 time steps
        #if X_pred.shape[1] > 5:
         #   X_pred = X_pred[:, -5:, :]

        # Load the latest model from MLflow
        mlflow.set_tracking_uri("/content/mlruns")
        get_custom_objects().update({"Attention": Attention})
        client = MlflowClient()
        runs = client.search_runs(experiment_ids=["0"], order_by=["start_time DESC"], max_results=1)

        if not runs:
            raise HTTPException(status_code=500, detail="No MLflow runs found.")

        latest_run = runs[0]
        model_uri = f"runs:/{latest_run.info.run_id}/model"
        print(model_uri)
        model = mlflow.keras.load_model(model_uri, custom_objects={'Attention': Attention})

        # Predict for the next 7 days
        predictions = []
        for i in range(7):
            print(f"Iteration {i}")
            prediction = model.predict(X_pred)

            # Inverse scale the prediction
            prediction_original = scaler.inverse_transform(
                np.hstack([np.zeros((len(prediction), 1)), prediction])
            )[:, -1]

            # Append the single prediction to the list
            predictions.append(float(prediction_original[0]))

            # Update X_pred with the new prediction
            new_data = np.array([[prediction_original[0]]])
            X_pred = np.append(X_pred[:, 1:, :], new_data.reshape(1, 1, 1), axis=1)



        # Generate prediction dates
        print(social_data['time'])
        last_date = social_data['time'].iloc[-1]
        print(last_date)
        prediction_dates = [last_date + timedelta(days=i) for i in range(1, 8)]

        # Prepare response
        result = [{"date": prediction_dates[i].strftime('%Y-%m-%d'), "predicted_stock_price": predictions[i]}
                  for i in range(len(predictions))]

        return {"predictions": result}



In [6]:

'''configuration Ngrok'''
# Install necessary packages: aiohttp for async subprocess execution and pyngrok for Ngrok integration

import os
import subprocess

# Set LD_LIBRARY_PATH to prioritize system NVIDIA libraries over built-in ones
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

# Define a helper function to run commands synchronously
def run(cmd):
    print('>>> starting', *cmd)
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

    # Process and print the output and error streams
    stdout, stderr = process.communicate()

    if stdout:
        print(stdout.decode('utf-8'))
    if stderr:
        print(stderr.decode('utf-8'))

# Replace this with your Ngrok authentication token
NGROK_AUTH_TOKEN = "2rPSxjtzWTP2vtpDDbK9AQf4VNy_6sdRogw7FGiJPcqgdQ2Pk"

# Authenticate with Ngrok using the token
run(['ngrok', 'config', 'add-authtoken', NGROK_AUTH_TOKEN])


>>> starting ngrok config add-authtoken 2rPSxjtzWTP2vtpDDbK9AQf4VNy_6sdRogw7FGiJPcqgdQ2Pk
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml



In [7]:
'''Création Ngrok tunnel '''
import nest_asyncio
import uvicorn

from pyngrok import ngrok

#!ngrok config add-authtoken 2rPSxjtzWTP2vtpDDbK9AQf4VNy_6sdRogw7FGiJPcqgdQ2Pk
# Required to run FastAPI in Colab
nest_asyncio.apply()
# Expose the server
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")
# Start the server
uvicorn.run(app, host="0.0.0.0", port=8000)



Public URL: NgrokTunnel: "https://17c2-35-221-215-83.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [391]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [391]
